In [3]:
import util


examples = util.load_example()
print(examples)

[{'question': '다음 내용들로 명훈 스타일 공지를 작성해주세요. \n            날짜 : 1월 24일 \n            시간 : 없음\n            컨텐츠 : 휴방 \n            할 말 : 산악회 로아 3막 레이드 완료', 'answer': '[공지] 1월 24일 명예훈장 방송 공지입니다! \n        반갑습니다 사랑하는 턱붕이 여러분!! 으아 로아 이번 3막 엄청 빡셌다 ㅠㅠ!!!!  \n        그래도 2일만에 클리어 완료.... ㅋㅋㅋ 다들 응원해주셔서 감사합니다!  \n        전체 순위는 6등이네요  \n        후후 잔혈까지 나이스 +_+ 스펙업 한 보람이 있었습니다 도와주신분들 너무너무x999 감사합니다ㅎ!  \n        산악회 다들 고생많았다!!!  \n        저는 오늘 배터리 방전입니다 일좀 하면서 쉬도록 하겠습니다 ㅎ  \n        유튜브 또 올라왔으니 재밌게 봐주시고 다시보기 채널도 많이 사랑해주세요!  \n        항상 시청해주셔서 감사합니다! 좋은 저녁 보내세요!'}, {'question': '다음 내용들로 명훈 스타일 공지를 공지 작성해주세요.  \n            날짜 : 1월 23일  \n            시간 : 3시, 7시  \n            컨텐츠 : 명조 광고, 로아 3막 트라이  \n            할 말 : 3시 명조 광고, 26시간 진행한 로아 3막 리트라이', 'answer': '[공지] 반갑습니다 1월 23일 방송 공지입니다! 3시 - 명조 광고 / 7시 로아 산악회 리트라이 가자!  \n        반갑습니다 사랑하는 턱붕이 여러분!! 1월 23일 방송 공지입니다!  \n        오우 어제 26시간 트라이....하고 잠깐 자고 왔습니다ㅋㅋㅋ 아직 3막을 못 깼다.....  \n        오늘 아침에 갑자기 정상화 패치를 해서 후반 진도를 나가야되는데 초반 진도를 나가게 되었습니다 크흡..  \

In [5]:
import os
import mykey
os.environ['OPENAI_API_KEY'] = mykey.my_openai()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [ ]:
example_prompt = PromptTemplate.from_template(
    "Question: n{question} nAnswer: n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: \n {question} \n Answer:",
    input_variables=["question"],
)

In [7]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('namuwiki.txt')
docs = loader.load()
docs

[Document(metadata={'source': 'namuwiki.txt'}, page_content="닉네임인 명예훈장을 줄인 명훈[9] 혹은 '띵훈'으로 잘 알려져있다. 버블파이터와 오버워치의 프로게이머로 활동했다. 닉네임의 유래는 버블파이터에서 게임 결과창에 나오는 훈장 시스템을 보고 만들었다고 한다.\n\n버블파이터 내에서 오랜기간 1위를 유지 했었던 유명한 랭커 출신으로, 아프리카TV 최초로 버블파이터 베스트BJ에 도달했다.[10] 2015년 10월쯤 트레져헌터에 입사하고, 프로게이머 BJ 칭호를 받았다. 현재는 카론 크리에이티브에 소속되어있다.\n\n오버워치에서는 겐지, 리퍼 플레이로 대한민국 서버 1위와 전세계 랭킹 6위[11]를 달성하는 엄청난 저력을 보이며 천상계 랭커 자리를 유지했으며, 대한민국 3대 겐지[명단]라는 이명으로 불리면서 오버워치를 플레이하는 유저들이라면 익히 한번씩은 들어봤을만한 인물들 중 한명이다.\n\n지인들과 결성한 'CONBOX T6'로 오버워치 프로게이머로 데뷔했으며, Lunatic-Hai에 재입단을 했지만 스트리밍에 본격적으로 집중하겠다는 자신의 굳은 의지로 팀을 나오게 되며 현재는 인터넷 방송인으로써 방송에서 다양한 게임을 플레이하고 있다.[13]\n\n아프리카TV에서 방송을 하고 있었으나 갑질사태를 기점으로 2016년 10월 26일 트위치로 방송국을 이전하였고, 트위치가 트위치 대한민국 사업 철수 사건으로 철수를 하자, 유튜브와 치지직, 아프리카TV 3개의 플랫폼에서 동시송출을 했다. 2024년 8월 13일부터는 치지직 파트너 계약 및 단독 송출을 하는 중이다.\n")]

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
import mykey
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
embeddings = OpenAIEmbeddings()
split_docs = text_splitter.split_documents(docs)
db = FAISS.from_documents(split_docs, embeddings)
retreieved = db.as_retriever()
